# Cluster Analysis

It is used to group a set of object in clusters belonging to the same group.

It is an unsupervised technique used to assess if data is divided in specific groups. It can also be used to reduce the number of dimensions to the number of groups found in data.

It is important to assess clustering tendencies before and after clustering in order to avoid mistakenly-created clusters. 

Before applying cluster analysis:

* Use scatter plots to see if objects are grouped

* Icon plots

* Ordered dissimilarity matrix

* Statistical tests

The two most popular clustering methods families are:

* **Hierarchical:** Algorithms which recursively find nested clusters. It makes n-1 steps producing n-1 partitions of data.

* **Partitional:** Find all the cluster as a partition of data, without hierarchy.

Hierarchical algorithms find nested cluster:

* in a **bottom-up way (agglomerative)** starting from indipendent points, merging them into clusters.

* in a **top-down way (divisive)** starting froma single cluster, splitting it into smaller ones.

In order to split values into clusters, we use the notion of **degree of proximity** based on distance, which we consider as the Euclidean distance between two observation in a p-dimensional space.

$$d(x_i, x_j) = \sqrt{\Sigma_{h=1}^{p}(x_{ih}-x_{jh})^2}$$

The distance matrix is symmetric, has 0s on the diagonal, and for each position has the distance between the two correspondent elements in the dataset.

### Hierarchical Agglomerative Procedure

* Each unit constitutes a group. The distance matrix $D$ contains the distances between all groups.

* The two groups which have minimum distance are merged. We decrease the dimension of $D$ by one unit, computing the new distance with the new group.

* Iterate until a group of all elements is created.

### Distances between groups

* **Single linkage (nearest neighbour):** Minimum distance between units belonging to the two respective groups.

* **Complete linkage (furthest neighbour):** Maximal distance between units belonging to the two groups (equivalent to the diameter of the smallest circle containing all the units of the two groups).

* **Average linkage**: Arithmetic mean of distance between all units of the two groups.

* **Centroid**: Distance between groups is given by the distance between centroids of the two groups. In the bottom-up approach, the increase in distance could be non-increasing with this distance.

* **Ward**: It is based on the decomposition of total deviance (sum of squared residuals of all units) into deviance within groups $D_W$ and deviance between groups $D_B$. The method merges iteratively the two groups which minimize the increase in withing deviance, maximising the increase in between deviance.

$$D_W - \Sigma_{g=1}^k\Sigma_{i\in g}((x_{ig}-\bar{x_g})^2$$

The **dendrogram** is the graphical representation of a hierarchical bottom-up method.

In [ ]:
The distance